In [ ]:
"""
Load the data from each year's CSV
"""

import pandas as pd

dtypes = {
    'C_MNTH': str,
    'C_WDAY': str,
    'C_VEHS': str,
    'V_ID': str
} # Remove DtypeWarnings from df imports

df_2016 = pd.read_csv('data/y_2016_en.csv', dtype=dtypes)
df_2017 = pd.read_csv('data/y_2017_en.csv', dtype=dtypes)
df_2018 = pd.read_csv('data/y_2018_en.csv', dtype=dtypes)
df_2019 = pd.read_csv('data/2019_dataset_en.csv', dtype=dtypes)

df = pd.concat([df_2016, df_2017, df_2018, df_2019], axis=0)

df.head(10)

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
0,2016,1,1,11,2,2,32,1,1,5,...,1,2015,1,M,56,QQ,1,2,U,2337584
1,2016,1,1,11,2,2,32,1,1,5,...,1,2015,UU,U,UU,UU,U,UU,U,2337584
2,2016,1,1,11,2,2,32,1,1,5,...,1,2010,1,F,30,11,2,2,1,2337584
3,2016,1,1,11,2,2,32,1,1,5,...,1,2010,2,U,1,QQ,N,NN,U,2337584
4,2016,1,1,16,2,2,32,1,2,5,...,1,2007,1,F,27,11,2,2,1,2337585
5,2016,1,1,16,2,2,32,1,2,5,...,1,2007,2,F,5,33,1,2,2,2337585
6,2016,1,1,16,2,2,32,1,2,5,...,1,2009,1,M,18,11,2,2,1,2337585
7,2016,1,1,20,2,1,4,1,2,2,...,1,2011,1,M,38,11,2,2,1,2337589
8,2016,1,1,20,2,1,4,1,2,2,...,NN,NNNN,UU,U,UU,UU,U,NN,3,2337589
9,2016,1,1,20,2,1,4,1,2,2,...,NN,NNNN,UU,U,UU,UU,U,NN,3,2337589


In [ ]:
"""
Shape the datasets into the form we can use
"""

# Drop the columns
df = df.drop(columns=['P_SEX','P_AGE','P_PSN','P_USER','P_SAFE'])

# Remove rows where 'P_ID' column contains 'UU' or 'NN'
df = df[~df['P_ID'].isin(['UU', 'NN'])]

# Reset the index to make the rows sequential
df.reset_index(drop=True, inplace=True)

# Group each entry by case number and the in-case vehicle ID
gk = df.groupby(['C_CASE','V_ID'])

# Combine all individual vehicles into single entry
# With this grouping, all case (C_*) and vehicle (V_*) columns will be identical across each group
squished_df = gk.apply(lambda g: g.iloc[0], include_groups=False).drop(columns=['P_ID','P_ISEV'])

# Count total numbers of people per vehicle
squished_df['P_COUNT'] = gk.P_ID.count()

# Count numbers of injuries and fatalities per group
vc_isev = gk.P_ISEV.value_counts()
squished_df['COUNT_INJURY']   = vc_isev.loc[pd.IndexSlice[:,:,'2']].reindex(squished_df.index, fill_value=0)
squished_df['COUNT_FATALITY'] = vc_isev.loc[pd.IndexSlice[:,:,'3']].reindex(squished_df.index, fill_value=0)

# Split into training and testing datasets
train_df = squished_df[squished_df['C_YEAR'] < 2019]
test_df  = squished_df[squished_df['C_YEAR'] >= 2019]

# Split train and test into X and ys
X_train, y_train = train_df.iloc[:, :-2], train_df.iloc[:, -2:]
X_test,  y_test  = test_df.iloc[:, :-2], test_df.iloc[:, -2:]

# Display the results
display(X_train.head())
y_train.head()

C_YEAR C_MNTH C_WDAY C_HOUR  C_SEV C_VEHS C_CONF C_RCFG C_WTHR  \
C_CASE  V_ID                                                                   
2337570 1       2016      1      7     14      2      2     QQ      1      1   
        2       2016      1      7     14      2      2     QQ      1      1   
2337571 1       2016      1      2      9      2      2      2      1      1   
        2       2016      1      2      9      2      2      2      1      1   
2337572 1       2016      1      6     10      2      2     32      1      1   

             C_RSUR C_RALN C_TRAF V_TYPE V_YEAR  P_COUNT  
C_CASE  V_ID                                              
2337570 1         1      1     18      1   2010        2  
        2         1      1     18      1   2010        1  
2337571 1         1      1     18      1   2014        1  
        2         1      1     18      1   2015        1  
2337572 1         1      1     UU      1   2014        1

COUNT_INJURY  COUNT_FATALITY
C_CASE  V_ID                              
2337570 1                0               0
        2                1               0
2337571 1                0               0
        2                1               0
2337572 1                1               0

Data validation

In [ ]:
import xgboost as xgb

w = None # TODO

dtrain = xgb.DMatrix(data=X_train, label=y_train, missing=0, weight=w)
dtest = xgb.DMatrix(data=X_test, label=y_test, missing=0, weight=w)

params = {
    'booster':'gblinear',
    'objective':'reg:linear'
}

xgb.train(params, dtrain, num_boost_round=10)

In [ ]:
xgb.cv(params, dtrain, num_boost_round=10, nfold=10)